In [ ]:
import glob
import os
import re
import pathlib
import logging
import math
import warnings

import pyFAI
import pyFAI.azimuthalIntegrator
from pyFAI.gui import jupyter
import fabio
import numpy as np
import matplotlib.pyplot as plt

[TODO] Setup calibration using pyFAI

# Visualisation of patterns
## Calibration image
The calibration image can be opened and read into an array.

The array can be directly plotted.

In [ ]:
# write the calibration file to an image array
image = fabio.open('calibration/DLS_CeO2_1200mm.tif')
print("image:", image)

calibration_image_array = image.data
print("calibration_image_array:", type(calibration_image_array), calibration_image_array.shape, calibration_image_array.dtype)

jupyter.display(calibration_image_array)

## Pattern image
The pattern file can be opened in the same way.

In [ ]:
# write the pattern file to an image array
image = fabio.open("data/pixium_03150.tif")

pattern_image_array = image.data
print("pattern_image_array:", type(pattern_image_array), pattern_image_array.shape, pattern_image_array.dtype)

jupyter.display(pattern_image_array)

Because the data is in arrays, we can do things like subtract the image. This could be useful for subtracting a dark image of the detector, for example.

In [ ]:
example_subtraction = pattern_image_array - calibration_image_array
jupyter.display(example_subtraction)

# Example workflows for data processing
First, we need to load the calibration, which contains information about our beamline setup, which we will use to perform an azimuthal integration or caking to the rest of our as-yet 'uncalibrated' data. To load an `azimuthal integrator` object or `ai` we use [pyFAI](https://pyfai.readthedocs.io/en/latest/).

*Notes on creating a .poni calibration file using [Dioptas](http://www.clemensprescher.com/programs/dioptas) are found on the [LightForm Wiki](https://lightform-group.github.io/wiki/tutorials/sxrd-caking-dioptas)*

In [ ]:
ai = pyFAI.load("calibration/DLS_CeO2_1200mm.poni")
print("\nIntegrator: \n", ai)

## Azimuthal integration

To perform an azimuthal integration we use the `integrate1d` function. 

There is also an `integrate2d` function, which is designed for caking of the data.

[TODO] - review all available options and determine those needed for our Diamond and DESY data.

* The number of points in 2-theta is defined by the user.
* The azimuthal range runs from -180 to 180, or -pi to pi, rather than 0 to 360 as in DAWN. An error will appear if using azimuth_range = (0,360).
* An output .dat file can be saved, which contains a header of metadata. 
* The result is returned as a numpy array of 2-theta and intensity.

An azimuthal integration can be performed like this.

In [ ]:
result = ai.integrate1d(pattern_image_array,
                        npt=10000,
                        azimuth_range=(-180,180),
                        unit="2th_deg",
                        correctSolidAngle=True,
                        polarization_factor=0.99,
                        method='full_csr',
                        filename="analysis/integrated.dat")

print('Size of result numpy array =', np.shape(result))

## Caking data

It is possible to cake the data using a loop to iterate over segments and integrate them using `integrate1d`. 

In [ ]:
number_of_cakes = 10

step = 360 / number_of_cakes

data=[]

for cake_number in range(number_of_cakes):
    azimuth_low = -180 + step * cake_number
    azimuth_high = azimuth_low + step
    print("Caking range = ",azimuth_low, " to ",azimuth_high)
    result = ai.integrate1d(pattern_image_array,
                            npt=1000,
                            azimuth_range=(azimuth_low,azimuth_high),
                            unit="2th_deg",
                            polarization_factor=0.99,
                            method='full_csr')
    data.append(result) 
print("Data array shape = ",np.shape(data))

However, caking is easier and faster using the `integrate2d` function (as explained in the documentation [here](https://pyfai.readthedocs.io/en/latest/usage/tutorial/Introduction/introduction.html) and [here](https://pyfai.readthedocs.io/en/latest/usage/cookbook/integration_with_python.html)). 

Providing a file name saves the data to a file in '.edf' format. The result is returned as an `Integrate2dResult` which contains a Numpy array with the caked data.

In [ ]:
result = ai.integrate2d(pattern_image_array,
                        1000,
                        36,
                        azimuth_range=(-180,180),
                        unit="2th_deg",
                        polarization_factor=0.99,
                        method='full_csr',
                        filename = "analysis/integrated.edf")
print('The result is a class of 3 arrays, with shape: ', np.shape(result))
print('The 1st array contains the integrated intensity for each cake, with shape: ', np.shape(result.intensity))
print('The 2nd array contains the two-theta values, with shape: ', np.shape(result.radial))
print('The 3rd array contains the angle of the cakes, with shape: ', np.shape(result.azimuthal))
type(result)

We can open the '.edf' file using FabIO.

In [ ]:
cake = fabio.open("analysis/integrated.edf")
print(cake.header)
print("cake:", type(cake.data), cake.data.shape, cake.data.dtype)

## Cake rotation

The orientation of the cakes is explained in the accompanying `pyFAI-example-caking-with-rotation-for-xrdfit` notebook using an example image. 

- WEST is -180${^\circ}$ / +180${^\circ}$ 
- SOUTH is -90${^\circ}$
- EAST is 0${^\circ}$ / 360${^\circ}$
- NORTH is +90${^\circ}$

pyFAI cakes the data anticlockwise (from -180${^\circ}$ to 0${^\circ}$ to +180${^\circ}$) which is the opposite direction to DAWN. pyFAI also starts caking the data from the `West` (-180${^\circ}$) direction (compared to DAWN which starts caking from the East direction).

And, as shown by the result below, the minimum cake angle in pyFAI is -180${^\circ}$, and so caking will slice along the horizontal, meaning the cake is centred on -175${^\circ}$ cake for 10${^\circ}$ slices, rather than being centred on the horizontal.

In [ ]:
result.azimuthal

 We can centre the cake on the horizontal (and vertical) by applying a rotation to the detector.

In [ ]:
# Rotate the detector so that the cardinal direction is in the center of the first cake.
number_of_cakes = 36
first_cake_angle = 360 / number_of_cakes
ai.rot3 = (first_cake_angle / 2) * (math.pi / 180) # convert rotation to radians
print(ai.rot3)

And we can flip the intensity data to order the cakes clockwise rather than anticlockwise. This is included in the subsequent caking multiple images example. 

# Caking multiple images

To iterate through some images we can create a loop, create an array for the data and then save it as a text file.

This caked dataset is now saved in a format that we can use in [xrdfit](https://xrdfit.readthedocs.io/en/stable/) to analyse how the single peak profiles change over time.

In [ ]:
# Supress warnings when TIFFs are read
logging.getLogger("fabio.TiffIO").setLevel(logging.ERROR)

# user inputs
number_of_points = 10000
number_of_cakes = 36

# get a list of the files
image_list = sorted(pathlib.Path("data/").glob("pixium*"))

for image_path in image_list:
    # create empty array
    caked_data = np.zeros((number_of_cakes + 1, number_of_points))
    
    # create an image array and cake the data
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        image = fabio.open(image_path)
    pattern_image_array = image.data
    result2d = ai.integrate2d(pattern_image_array,
                            number_of_points,
                            number_of_cakes,
                            unit="2th_deg",
                            polarization_factor=0.99,
                            method='full_csr')
    
    # flip the intensity data to order cakes clockwise rather than anticlockwise
    intensity = np.flip(result2d.intensity.T, axis=1)
    
    # reshape radial labels to 2D array so they can be attached to the intensity data.
    radial = np.reshape(result2d.radial, (-1, 1))
    
    result_array = np.hstack((radial, intensity))
    
    # write out the caked data to a text file
    output_path = f"analysis/{image_path.stem}.dat"
    np.savetxt(output_path, result_array)

In [ ]:
plt.plot(result_array[:,0],result_array[:,2], marker = ".")
plt.xlim(3,4);